# Data Structures: Graphs & Dependency Resolution

Master graph algorithms for pipeline execution ordering.

## 🎯 The Problem

**Pipeline nodes must execute in dependency order:**
- Node C depends on Node A and Node B
- Node B depends on Node A
- What order should they execute?

**Answer**: A → B → C

**Bonus**: Can any nodes run in parallel?

**Answer**: No, but if D also depends on A, then B and D can run in parallel:
- Layer 1: [A]
- Layer 2: [B, D]
- Layer 3: [C]

## 🦉 First Principles

### Graph Fundamentals

**Graph**: G = (V, E) where V = vertices, E = edges

**Directed Graph**: Edges have direction (A → B)

**DAG (Directed Acyclic Graph)**: Directed graph with no cycles
- Perfect for dependencies
- If A → B → A, we have a cycle (circular dependency)

### Adjacency List vs Matrix

**Adjacency List**: `{node: [neighbors]}`
- Space: O(V + E)
- Check edge: O(degree)
- Iterate neighbors: O(degree)
- Best for sparse graphs

**Adjacency Matrix**: 2D array[i][j] = 1 if edge exists
- Space: O(V²)
- Check edge: O(1)
- Iterate neighbors: O(V)
- Best for dense graphs

**For dependencies**: Adjacency list (graphs are sparse)

## ⚡ Example: Build a Simple Graph

In [ ]:
from typing import List, Dict, Set
from collections import defaultdict, deque

class Graph:
    """Simple directed graph using adjacency list."""
    
    def __init__(self):
        self.adjacency_list: Dict[str, List[str]] = defaultdict(list)
        self.nodes: Set[str] = set()
    
    def add_node(self, node: str) -> None:
        """Add a node to the graph."""
        self.nodes.add(node)
    
    def add_edge(self, from_node: str, to_node: str) -> None:
        """Add directed edge from_node -> to_node."""
        self.nodes.add(from_node)
        self.nodes.add(to_node)
        self.adjacency_list[from_node].append(to_node)
    
    def get_neighbors(self, node: str) -> List[str]:
        """Get all neighbors of a node."""
        return self.adjacency_list[node]
    
    def __repr__(self) -> str:
        lines = ["Graph:"]
        for node in sorted(self.nodes):
            neighbors = self.adjacency_list[node]
            if neighbors:
                lines.append(f"  {node} → {neighbors}")
            else:
                lines.append(f"  {node}")
        return "\n".join(lines)

# Build example graph
g = Graph()
g.add_edge("A", "B")
g.add_edge("A", "C")
g.add_edge("B", "D")
g.add_edge("C", "D")

print(g)

## 🔍 Graph Traversal

### Depth-First Search (DFS)
- Go deep before going wide
- Uses stack (or recursion)
- Good for cycle detection, topological sort

In [ ]:
def dfs(graph: Graph, start: str) -> List[str]:
    """Depth-first search."""
    visited = set()
    result = []
    
    def visit(node: str):
        if node in visited:
            return
        
        visited.add(node)
        result.append(node)
        
        for neighbor in graph.get_neighbors(node):
            visit(neighbor)
    
    visit(start)
    return result

print("DFS from A:", dfs(g, "A"))

### Breadth-First Search (BFS)
- Go wide before going deep
- Uses queue
- Good for shortest path, level-order traversal

In [ ]:
def bfs(graph: Graph, start: str) -> List[str]:
    """Breadth-first search."""
    visited = {start}
    queue = deque([start])
    result = []
    
    while queue:
        node = queue.popleft()
        result.append(node)
        
        for neighbor in graph.get_neighbors(node):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return result

print("BFS from A:", bfs(g, "A"))

## 🔄 Cycle Detection

**Why?** Circular dependencies are errors in pipelines.

**Algorithm**: DFS with recursion stack
- **visited**: Nodes fully processed
- **rec_stack**: Nodes in current path
- **Cycle**: Node found in rec_stack

In [ ]:
def has_cycle(graph: Graph) -> tuple[bool, List[str] | None]:
    """Detect if graph has a cycle.
    
    Returns:
        (has_cycle, cycle_path)
    """
    visited = set()
    rec_stack = set()
    
    def visit(node: str, path: List[str]) -> List[str] | None:
        if node in rec_stack:
            # Found cycle - return the cycle path
            cycle_start = path.index(node)
            return path[cycle_start:] + [node]
        
        if node in visited:
            return None
        
        visited.add(node)
        rec_stack.add(node)
        path.append(node)
        
        for neighbor in graph.get_neighbors(node):
            cycle = visit(neighbor, path[:])
            if cycle:
                return cycle
        
        rec_stack.remove(node)
        return None
    
    for node in graph.nodes:
        if node not in visited:
            cycle = visit(node, [])
            if cycle:
                return True, cycle
    
    return False, None

# Test with acyclic graph
has_cycle_result, cycle = has_cycle(g)
print(f"Has cycle: {has_cycle_result}")

# Test with cyclic graph
g_cyclic = Graph()
g_cyclic.add_edge("A", "B")
g_cyclic.add_edge("B", "C")
g_cyclic.add_edge("C", "A")  # Creates cycle

has_cycle_result, cycle = has_cycle(g_cyclic)
print(f"\nCyclic graph has cycle: {has_cycle_result}")
print(f"Cycle path: {cycle}")

## 📊 Topological Sort

**Definition**: Linear ordering where for every edge u→v, u comes before v

**Use case**: Dependency ordering for execution

**Kahn's Algorithm** (BFS-based):
1. Calculate in-degree (# incoming edges) for each node
2. Add all in-degree=0 nodes to queue
3. Process queue:
   - Remove node
   - Add to result
   - Decrease in-degree of neighbors
   - If neighbor in-degree becomes 0, add to queue
4. If all nodes processed → valid topological sort
5. If not all processed → cycle exists

In [ ]:
def topological_sort(graph: Graph) -> List[str]:
    """Kahn's algorithm for topological sorting.
    
    Returns:
        Nodes in dependency order
        
    Raises:
        ValueError: If cycle detected
    """
    # Calculate in-degree for each node
    in_degree = {node: 0 for node in graph.nodes}
    for node in graph.nodes:
        for neighbor in graph.get_neighbors(node):
            in_degree[neighbor] += 1
    
    # Queue nodes with in-degree 0
    queue = deque([node for node, degree in in_degree.items() if degree == 0])
    result = []
    
    while queue:
        node = queue.popleft()
        result.append(node)
        
        # Reduce in-degree for neighbors
        for neighbor in graph.get_neighbors(node):
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    # Check if all nodes processed
    if len(result) != len(graph.nodes):
        raise ValueError("Graph has a cycle - cannot perform topological sort")
    
    return result

# Test
sorted_nodes = topological_sort(g)
print("Topological order:", sorted_nodes)

# Verify: for each edge, from_node comes before to_node
for node in graph.nodes:
    node_idx = sorted_nodes.index(node)
    for neighbor in g.get_neighbors(node):
        neighbor_idx = sorted_nodes.index(neighbor)
        assert node_idx < neighbor_idx, f"{node} should come before {neighbor}"
print("✓ Valid topological sort")

## 🏗️ Build: DependencyGraph

Now build a production-grade dependency graph like Odibi's.

In [ ]:
from typing import List, Dict, Set, Optional
from collections import defaultdict, deque
from dataclasses import dataclass

@dataclass
class Node:
    """Pipeline node configuration."""
    name: str
    depends_on: List[str]


class DependencyGraph:
    """Dependency graph for pipeline execution."""
    
    def __init__(self, nodes: List[Node]):
        self.nodes = {node.name: node for node in nodes}
        # Forward adjacency: dependency -> dependent
        self.adjacency_list: Dict[str, List[str]] = defaultdict(list)
        # Reverse adjacency: dependent -> dependencies
        self.reverse_adjacency_list: Dict[str, List[str]] = defaultdict(list)
        
        self._build_graph()
        self._validate()
    
    def _build_graph(self) -> None:
        """Build adjacency lists."""
        for node in self.nodes.values():
            for dependency in node.depends_on:
                # Edge: dependency -> node
                self.adjacency_list[dependency].append(node.name)
                self.reverse_adjacency_list[node.name].append(dependency)
    
    def _validate(self) -> None:
        """Validate graph."""
        self._check_missing_dependencies()
        self._check_cycles()
    
    def _check_missing_dependencies(self) -> None:
        """Check all dependencies exist."""
        for node in self.nodes.values():
            for dep in node.depends_on:
                if dep not in self.nodes:
                    raise ValueError(
                        f"Node '{node.name}' depends on '{dep}' which doesn't exist"
                    )
    
    def _check_cycles(self) -> None:
        """Detect circular dependencies."""
        visited = set()
        rec_stack = set()
        
        def visit(node: str, path: List[str]) -> Optional[List[str]]:
            if node in rec_stack:
                cycle_start = path.index(node)
                return path[cycle_start:] + [node]
            
            if node in visited:
                return None
            
            visited.add(node)
            rec_stack.add(node)
            path.append(node)
            
            for dependent in self.adjacency_list[node]:
                cycle = visit(dependent, path[:])
                if cycle:
                    return cycle
            
            rec_stack.remove(node)
            return None
        
        for node_name in self.nodes.keys():
            if node_name not in visited:
                cycle = visit(node_name, [])
                if cycle:
                    raise ValueError(f"Circular dependency: {' -> '.join(cycle)}")
    
    def topological_sort(self) -> List[str]:
        """Return nodes in execution order."""
        in_degree = {name: len(node.depends_on) for name, node in self.nodes.items()}
        queue = deque([name for name, degree in in_degree.items() if degree == 0])
        result = []
        
        while queue:
            node = queue.popleft()
            result.append(node)
            
            for dependent in self.adjacency_list[node]:
                in_degree[dependent] -= 1
                if in_degree[dependent] == 0:
                    queue.append(dependent)
        
        if len(result) != len(self.nodes):
            raise ValueError("Failed topological sort (cycle detected)")
        
        return result
    
    def get_execution_layers(self) -> List[List[str]]:
        """Group nodes into parallel execution layers."""
        in_degree = {name: len(node.depends_on) for name, node in self.nodes.items()}
        layers = []
        remaining = set(self.nodes.keys())
        
        while remaining:
            # Find all nodes with in-degree 0
            current_layer = [name for name in remaining if in_degree[name] == 0]
            
            if not current_layer:
                raise ValueError("Cannot create execution layers (cycle detected)")
            
            layers.append(current_layer)
            
            # Remove layer and update in-degrees
            for node in current_layer:
                remaining.remove(node)
                for dependent in self.adjacency_list[node]:
                    if dependent in remaining:
                        in_degree[dependent] -= 1
        
        return layers
    
    def get_dependencies(self, node_name: str) -> Set[str]:
        """Get all dependencies (transitive) for a node."""
        if node_name not in self.nodes:
            raise ValueError(f"Node '{node_name}' not found")
        
        dependencies = set()
        queue = deque([node_name])
        
        while queue:
            current = queue.popleft()
            for dep in self.reverse_adjacency_list[current]:
                if dep not in dependencies:
                    dependencies.add(dep)
                    queue.append(dep)
        
        return dependencies


# Test
nodes = [
    Node("extract", []),
    Node("transform", ["extract"]),
    Node("validate", ["extract"]),
    Node("load", ["transform", "validate"]),
]

graph = DependencyGraph(nodes)
print("Execution order:", graph.topological_sort())
print("\nExecution layers:")
for i, layer in enumerate(graph.get_execution_layers(), 1):
    print(f"  Layer {i}: {layer}")

print("\nDependencies of 'load':", graph.get_dependencies("load"))

## ✅ Test: Validation

In [ ]:
# Test 1: Valid graph
try:
    graph = DependencyGraph([
        Node("A", []),
        Node("B", ["A"]),
        Node("C", ["B"]),
    ])
    print("✓ Valid graph created")
except ValueError as e:
    print(f"✗ Failed: {e}")

# Test 2: Missing dependency
try:
    graph = DependencyGraph([
        Node("A", []),
        Node("B", ["C"]),  # C doesn't exist
    ])
    print("✗ Should have detected missing dependency")
except ValueError as e:
    print(f"✓ Caught missing dependency: {e}")

# Test 3: Circular dependency
try:
    graph = DependencyGraph([
        Node("A", ["B"]),
        Node("B", ["C"]),
        Node("C", ["A"]),  # Cycle!
    ])
    print("✗ Should have detected cycle")
except ValueError as e:
    print(f"✓ Caught circular dependency: {e}")

# Test 4: Execution layers
graph = DependencyGraph([
    Node("A", []),
    Node("B", []),
    Node("C", ["A"]),
    Node("D", ["A"]),
    Node("E", ["C", "D"]),
])
layers = graph.get_execution_layers()
assert layers == [["A", "B"], ["C", "D"], ["E"]] or layers == [["B", "A"], ["C", "D"], ["E"]]
print("✓ Execution layers correct")

## 🎓 Summary

You've learned:
1. **Graph representations** - Adjacency list for sparse graphs
2. **Traversal algorithms** - DFS for cycles, BFS for layers
3. **Topological sort** - Kahn's algorithm for dependency order
4. **Execution layers** - Group independent nodes for parallelism
5. **Production patterns** - Validation, bidirectional adjacency

Next: Practice with [exercises.ipynb](exercises.ipynb)